In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import os
import cv2

In [2]:
def ELA(image_path, quality=95):
    temp_path = "temp.jpg"
    original_image = cv2.imread(image_path)
    cv2.imwrite(temp_path, original_image, [cv2.IMWRITE_JPEG_QUALITY, quality])
    temp_image = cv2.imread(temp_path)
    ELA_image = cv2.absdiff(original_image, temp_image)
    ELA_image = ELA_image.astype(np.uint8)
    os.remove(temp_path)
    return ELA_image

In [3]:
def load(real_dir, fake_dir, img_size = (128, 128)):
    real, fake = [], []
    
    for image in os.listdir(real_dir):
        image_path = os.path.join(real_dir, image)
        #image = cv2.imread(image_path)
        ELA_image = ELA(image_path)
        ELA_image = cv2.resize(ELA_image, img_size)
        real.append(ELA_image)
        
    for image in os.listdir(fake_dir):
        image_path = os.path.join(fake_dir, image)
        #image = cv2.imread(image_path)
        ELA_image = ELA(image_path)
        ELA_image = cv2.resize(ELA_image, img_size)
        fake.append(ELA_image)
        
    real_label = [1] * len(real)
    fake_label = [0] * len(fake)
    
    images = np.array(real + fake)
    labels = np.array(real_label + fake_label)
    return images, labels

In [5]:
real_dir = "/kaggle/input/data-images-1/Deepfake/Real"
fake_dir = "/kaggle/input/data-images-1/Deepfake/Fake"
images, labels = load(real_dir, fake_dir)

#images

In [6]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train, x_test = x_train.astype('float32') / 255, x_test.astype('float32') / 255
input_shape = x_train.shape[1:]

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

In [8]:
def adjust_learning_rate(epochs):
  learning_rate = 1e-1
  if epochs > 160:
    learning_rate *= 5e-5
  elif epochs > 120:
    learning_rate *= 1e-4
  elif epochs > 80:
    learning_rate *= 5e-4
  elif epochs > 40:
    learning_rate *= 5e-3
  elif epochs >= 0:
    learning_rate *= 1e-2
  return learning_rate

In [9]:
def create_layer(x, filters = 16, kernel_size = 3,
                 strides=1, activation='relu',
                 batch_nomarlization = True,
                 first_layer = True):
  conv = Conv2D(filters, kernel_size, strides=strides,
                padding='same',
                kernel_initializer = 'he_normal',
                kernel_regularizer=l2(1e-3))
  input = x
  if first_layer:
    input = conv(x)
    if batch_nomarlization:
      input = BatchNormalization()(input)
    if activation is not None:
      input = Activation(activation)(input)
  else:
    if batch_nomarlization:
      input = BatchNormalization()(input)
    if activation is not None:
      input = Activation(activation)(input)
    input = conv(input)
  return input

In [10]:
def resnet(input_shape, depth, num_classes=2):
  num_filters = 16
  num_blocks = int((depth - 2) / 6)

  inputs = Input(shape=input_shape)
  x = create_layer(inputs)

  for blocks in range(3):
    for i in range(num_blocks):
      strides = 1
      if blocks > 0 and i == 0:
        strides = 2
      y = create_layer(x, num_filters, strides=strides)
      y = create_layer(y, filters=num_filters, strides=strides, activation=None)
      if blocks > 0 and num_blocks == 0:
        x = create_layer(x, filters=num_filters, kernel_size=1,
                         strides=strides,
                         activation=None,
                         batch_normalization=False)
      x = keras.layers.add([x, y])
      x = Activation('relu')(x)
    num_filters *= 2

    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    results = Dense(num_classes, activation='softmax',
                    kernel_initializer='he_normal')(y)
    model = Model(inputs=inputs, outputs=results)
    return model

In [11]:
model = resnet(input_shape, 152)
lr_scheduler = LearningRateScheduler(adjust_learning_rate)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=5e-6)
model_type = 'resnet152'
save_dir = os.path.join(os.getcwd(), 'saved_models') 
model_name = f'Casia1_{model_type}_model.{{epoch:03d}}.keras'
if not os.path.isdir(save_dir): 
    os.makedirs(save_dir) 
filepath = os.path.join(save_dir, model_name)

checkpoint = ModelCheckpoint(filepath=filepath, 
                              monitor='val_accuracy', 
                              verbose=1, 
                              save_best_only=True) 
callbacks = [checkpoint, lr_reducer, lr_scheduler]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=adjust_learning_rate(0)),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_test, y_test),
          shuffle=True, callbacks=callbacks)

Epoch 1/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5736 - loss: 4.5563   
Epoch 1: val_accuracy improved from -inf to 0.50408, saving model to /kaggle/working/saved_models/Casia1_resnet152_model.001.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - accuracy: 0.5745 - loss: 4.5255 - val_accuracy: 0.5041 - val_loss: 2.3616 - learning_rate: 0.0010
Epoch 2/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 775ms/step - accuracy: 0.7423 - loss: 2.2068
Epoch 2: val_accuracy did not improve from 0.50408
31/31 ━━━━━━━━━━━━━━━━━━━━ 25s 820ms/step - accuracy: 0.7421 - loss: 2.2063 - val_accuracy: 0.5041 - val_loss: 2.5230 - learning_rate: 0.0010
Epoch 3/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 813ms/step - accuracy: 0.7945 - loss: 2.0336
Epoch 3: val_accuracy did not improve from 0.50408
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 859ms/step - accuracy: 0.7944 - loss: 2.0339 - val_accuracy: 0.5041 - val_loss: 2.2731 - learning_rate: 0.0010
Epoch 4/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 838ms/step - accuracy: 0.8500 - loss: 1.9

In [10]:
model.evaluate(x_test, y_test, verbose=2)

11/11 - 3s - 302ms/step - accuracy: 0.8000 - loss: 2.1479


[2.147914171218872, 0.800000011920929]

In [11]:
#import shutil
#shutil.rmtree("/kaggle/working/saved_models (ELA)")

In [12]:
save_dir = os.path.join(os.getcwd(), 'saved_models')

'''models = []
for path in os.listdir(save_dir):
    model_dir = os.path.join(save_dir, path)
    models.append(load_model(model_dir))
best_model = models[-1]'''
best_model_path = os.path.join(save_dir, "/kaggle/working/saved_models/Casia1_resnet152_model.097.keras")
best_model = load_model(best_model_path)

In [13]:
y_pred = best_model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_class))

16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 234ms/step
              precision    recall  f1-score   support

           0       0.78      0.78      0.78       243
           1       0.78      0.79      0.78       247

    accuracy                           0.78       490
   macro avg       0.78      0.78      0.78       490
weighted avg       0.78      0.78      0.78       490



In [14]:
Real_dir = "/kaggle/input/testing-images/Testing/Real"
Fake_dir = "/kaggle/input/testing-images/Testing/Fake"
testing_images, testing_labels = load(Real_dir, Fake_dir)
#images

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(testing_images, testing_labels, test_size=0.1, random_state=42)
X_train, X_test = X_train.astype('float32') / 255, X_test.astype('float32') / 255

Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [16]:
y_predict = best_model.predict(X_train)
y_predict_class = np.argmax(y_predict, axis=1)
y_true = np.argmax(Y_train, axis=1)
print(classification_report(y_true, y_predict_class))

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 207ms/step
              precision    recall  f1-score   support

           0       0.95      0.77      0.85       160
           1       0.78      0.95      0.86       141

    accuracy                           0.85       301
   macro avg       0.86      0.86      0.85       301
weighted avg       0.87      0.85      0.85       301



In [18]:
def load1(dir, img_size = (128, 128)):
    images=[]
    
    for image in os.listdir(dir):
        image_path = os.path.join(dir, image)
        image = ELA(image_path)
        image = cv2.resize(image, img_size)
        images.append(image)
        
    
    images = np.array(images)
    return images

In [43]:
#model2 = load_model("/kaggle/input/model/keras/default/1/model.keras")
testing = load1("/kaggle/input/sample-test-2/images")
convert = testing.astype('float32') / 255

In [44]:
pred = best_model.predict(convert)
threshold = 0.7
predictions = (pred[:, 1] > threshold).astype(int)

print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0]


In [45]:
pred

array([[9.0953159e-01, 9.0468407e-02],
       [4.1026518e-01, 5.8973485e-01],
       [4.4154555e-01, 5.5845445e-01],
       [1.6203335e-02, 9.8379666e-01],
       [7.4446660e-01, 2.5553337e-01],
       [3.6394927e-01, 6.3605070e-01],
       [2.4333556e-01, 7.5666451e-01],
       [3.1168348e-01, 6.8831646e-01],
       [5.0756049e-01, 4.9243948e-01],
       [2.3612976e-01, 7.6387030e-01],
       [1.9646749e-01, 8.0353254e-01],
       [1.8644351e-01, 8.1355649e-01],
       [5.2270734e-01, 4.7729266e-01],
       [4.4854552e-01, 5.5145448e-01],
       [4.0399886e-04, 9.9959606e-01],
       [9.6959203e-01, 3.0407993e-02],
       [1.6895185e-01, 8.3104813e-01],
       [5.1013076e-01, 4.8986918e-01],
       [3.0770642e-01, 6.9229358e-01],
       [4.9142206e-01, 5.0857794e-01]], dtype=float32)

In [76]:
model = load_model("/kaggle/input/deepface/keras/default/1/DeepFace_XceptionNet_model.063.keras")
prediction = model.predict(convert)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[8.6975977e-04, 9.9913031e-01],
       [8.7008596e-04, 9.9912995e-01],
       [8.7002205e-04, 9.9912995e-01],
       [8.6958410e-04, 9.9913043e-01],
       [8.6984388e-04, 9.9913019e-01],
       [8.6972211e-04, 9.9913031e-01],
       [8.6985010e-04, 9.9913019e-01],
       [8.6977670e-04, 9.9913019e-01],
       [8.6991012e-04, 9.9913007e-01],
       [8.6959108e-04, 9.9913043e-01],
       [8.6982147e-04, 9.9913019e-01],
       [8.6977298e-04, 9.9913019e-01],
       [8.6999062e-04, 9.9913007e-01],
       [8.6979492e-04, 9.9913019e-01],
       [8.6979492e-04, 9.9913019e-01],
       [8.6992839e-04, 9.9913007e-01],
       [8.6966524e-04, 9.9913031e-01],
       [8.6993253e-04, 9.9913007e-01],
       [8.6986541e-04, 9.9913019e-01],
       [8.6990639e-04, 9.9913007e-01]], dtype=float32)